In [ ]:
import os

In [11]:
import os
import struct
import traceback
import pandas as pd
import numpy as np


def read_floats(data, n, size):
    return struct.unpack('d'*size, data[n:n + 8 * size])


class OPS: 
    def __init__(self):
        self.dirlist = None
        self.curfilename = None
        self.prefix = None

        self.model   = None
        self.sernum  = None
        self.sample_length = None

        self.dataheader = 'Sample #,Date,Start Time,Temp(C),Pressure(atm),Rel. Humidity,Errors,Alarm Triggered,Dilution Factor,Dead Time,Median,Mean,Geo. Mean,Mode,Geo. St. Dev.,Total Conc. '
        self.midheader = ['(µg/m³),Vol.Wt.Mean Diam.', '(µm²/cm³),Surf.Wt.Mean Diam.', '(#/cm³),Midpoint Diameter']

        self.header = None
        self.boundaries = None
        self.LB = None
        self.UB = None

        if 'ix' in os.name:
            self.sep = '/'  ## -- path separator for LINIX
        else:
            self.sep = '\\' ## -- path separator for Windows

        self.weights = ['Mass', 'Surface', 'Number']
        self.units   = ['dW/dDp', 'dW/dlogDp', 'Concentration (dW)', '% Concentration', 'Raw Counts']

        #self.dataheader = 'Sample #,Date,Start Time,Temp(C),Pressure(atm),Rel. Humidity,Errors,Alarm Triggered,Dilution Factor,Dead Time,Median,Mean,Geo. Mean,Mode,Geo. St. Dev.,Total Conc. '
        #self.midheader = ['(µg/m³),Vol.Wt.Mean Diam.', '(µm²/cm³),Surf.Wt.Mean Diam.', '(#/cm³),Midpoint Diameter']


    ## return sorted list of data files
    def get_dirlist(self, dirname):
        dirlist = [x for x in os.listdir(dirname) if x.split('.')[-1] == 'O30']

        ## сравнить префиксы у всех файлов O30
        ## префиксом считается часть имени файла до первой точки
        prefix = set(x.split('.')[0] for x in dirlist)
        #print(len(prefix), prefix)
        if len(prefix) == 1: 
            self.prefix = prefix.pop()
        elif len(prefix) > 1:
            print(f"В папке  {dirname} есть данные разных серий. Положите каждую серию в свою папку и запустите программу для каждой серии отдельно.")
            return []
        else:
            print(f"В папке {dirname} нет файлов с данными. Положите данные в папку и запустите программу снова.")
            return []

        ## отсортировать в порядке возрастания номеров
        dirlist = sorted(dirlist, 
                        key=lambda x: int(x.split('.')[-2]) if x.split('.')[1:-1] else 0)
        return dirlist


    def translate_header(self, header):
        model_byte = 64
        sernum_byte = model_byte + 20
        model  = header[model_byte:model_byte + 4].decode()
        sernum = header[sernum_byte:sernum_byte + 10].decode()
        sample_length_byte = 2170 * 4
        sample_length = struct.unpack('i', header[sample_length_byte:sample_length_byte + 4])[0]
        boundaries = read_floats(header, 8692, 17) ## Boundaries

        ## \todo сравнение заголовка текущего файла с заголовком предыдущего

        ## header 1
        header1 = '\n'.join([
                        f"Instrument Model,{model}",
                        f'Instrument Serial Number,{sernum}',
                        f'Sample Length (s),{sample_length}',
                        'Alarm Set,No',
                        'Dead Time Correction Applied,Yes'
                        ])

        header2 = '\n'.join([
                        f'Refractive Index Applied,No', '',
                        ',,,,,,,,,,,,,,,,LB,' + ','.join(map(str,boundaries[:-1])),
                        ',,,,,,,,,,,,,,,,UB,' + ','.join(map(str,boundaries[1:])),
                        ',,,,,,,,,,,,,,,,LB with RI,' + ','.join(map(str,boundaries[:-1])) + ',',
                        ',,,,,,,,,,,,,,,,UB with RI,' + ','.join(map(str,boundaries[1:])) + ','
                        ])

        self.header = [header1, header2]
        self.boundaries = boundaries[:]
        self.LB = boundaries[:-1]
        self.UB = boundaries[1:]
        self.dB = [(u - l) for l, u in zip(self.LB, self.UB)]
        self.model = model
        self.sernum = sernum
        self.sample_length = sample_length            


    def print_header(self, unit, weight):
        ## check unit and weight
        if unit > len(self.units) - 1 or weight > len(self.weights) - 1:
            print(f"Out of parameter range. Unexpected error in function {traceback.extract_stack()[-1][2]}")
            return 1
        ## Для Raw считаем только Number     
        if unit == self.units[-1]:
            weight = self.weights[-1]

        print(f"Sample file, {self.curfilename}")
        print(self.header[0])
        ## units
        print(f'Units,{self.units[unit]}')
        ## parameter
        print(f'Weight,{self.weights[weight]}')
        print(self.header[1])


    ## читать из данных события значения в каналах
    def read_channels(self, data):
        chan_len = 20 ## длина записи данных одного канала в байтах
        channels = []
        for i in range(15):
            ch = struct.unpack('dhhii', data[i * chan_len: (i + 1)* chan_len])
            #channels.append(ch)
            channels += ch
        return channels


    def generate_csv_header(self):
        text_header = ['lenght', 'datatime', 'date', 'time', 'x1', 'x2', 'x3'] + \
                    ['y'+str(i) for i in range(1, 21)] + ['x4', 'temperature', 'pressure', 'humidity']
        chan_header = [x + str(i)  for i in range(1,16) for x in ["dt" , 'tr', 'al', 'vl', 'nch'] ]
        text_header += chan_header
        text_header += ['m' + str(i) for i in range(1, 4)] # + ['TotalConc']
        text_header += ['z' + str(i) for i in range(0, 16)] + ['other']
        #text_header += [f"{(self.LB[i] + self.UB[i]) * 0.5:.3f}" for i in range(0, 16)] + ['other']
        ## tr14 - Errors:
        text_header[text_header.index('tr14')] = "Errors"
        ## dt14 - Dead Time - заменить в заголовке
        text_header[text_header.index('dt14')] = "Dead Time"
        return text_header


    def read_one_event(self, file_with_data):
        '''  Читаем одну запись  '''
        line_length = 658 ## длина одной строки данных
        chan_pos = 220    ## начало записи данных каналов в байтах
        chan_len = 300    ## длина записи данных всех каналов
        last_pos = 534 - 8    ## начало последних данных в записи, позиция первого байта после канало

        ## read binary data from file
        data_byte = file_with_data.read(line_length)
        
        ## check length of read line
        if len(data_byte) == 0:
            return

        if len(data_byte) < line_length:
            print(f"\nError in data!!!! len = {len(data_byte)} bytes but expected {line_length} bytes" )
            return

        ## check first two chars: if no 92 02 - no data
        if not data_byte.startswith(b'\x92' + b'\x02'):
            print(f"\nError in data!! No standart start bytes!!! No standart length of event data") ## \todo правильно обработать эту ошибку
            return
        
        ### read first bytes
        format_first = 'ii6hiiI' + 23 * 'd' +'f'
        firstdata = list(struct.unpack(format_first, data_byte[:chan_pos]))
        ## соберем дату
        firstdata.insert(2, str(firstdata[4]) + '-' + str(f'{firstdata[2]:02d}') + '-' + str(f'{firstdata[3]:02d}'))
        [firstdata.pop(3) for _ in range(3)]
        ## соберем время
        firstdata.insert(3, str(f'{firstdata[3]:02d}') + ':' + str(f'{firstdata[4]:02d}') + ':' + str(f'{firstdata[5]:02d}'))
        [firstdata.pop(4) for _ in range(3)]

        ### read channels 
        channels = self.read_channels(data_byte[chan_pos:])

        ### read 7 integers after channels
        #print(chan_pos + chan_len + 5 * 2, last_pos)
        middle  = list(struct.unpack('3H', data_byte[chan_pos + chan_len: last_pos]))
        #middle += list(struct.unpack('f',  data_byte[chan_pos + chan_len + 5 * 2: last_pos]))

        ### read last bytes
        last_num = struct.unpack('16dI', data_byte[last_pos:])

        alldata = list(firstdata) + list(channels) + list(middle) + list(last_num)
        return alldata


    def read_binary_data(self, file_with_data):
        ### make dataframe
        text_header = self.generate_csv_header()
        df = pd.DataFrame(columns=text_header)

        ## Читаем файл, пока считываются строки по 658 символов \todo заменить число на переменную
        event_length = 658
        n = 0
        reading = True
        while True:
            ## read one event
            alldata = self.read_one_event(file_with_data)
            if not alldata:
                break

            ### make dictionary
            newline = {x:y for x,y in zip(text_header, alldata)}
            df = pd.concat([df,pd.DataFrame([newline])], ignore_index=True)

        #print(firstdata, channels, middle, last_num, sep='\n-----------------\n')
        #print(alldata)
        #print(last_num)
        #df['dt14']
        #print(df[['date', 'time', 'Dead Time']])
        df.to_csv(self.curfilename + ".txt", sep=' ') 
        df.to_csv(self.curfilename + ".csv")

        return df    


    ## translate all data files from directory in path
    def translate_data(self, path):
        self.dirlist = self.get_dirlist(path)
        if not self.dirlist:
            return 1

        ## перебрать все файлы
        for filename in self.dirlist[:2]:
            print("===================")
            self.curfilename = filename
            self.translate_ops_file(path + filename)


    ## translate one O30 file
    def translate_ops_file(self, filename):
        file_handler = open(filename, "rb")
        
        ## read header
        header = file_handler.read(9172)
        header = self.translate_header(header)
        self.print_header(4,2)

        ## read binary data
        data = self.read_binary_data(file_handler)

        ## 
        ## particle diameter (channel midpoint)
        LB = pd.Series(self.LB)
        UB = pd.Series(self.UB)
        ## particle diameter (channel midpoint)
        self.Dp = (LB + UB) / 2
        self.dDp = UB - LB
        self.Dps = LB * ((1 + (UB/LB) + (UB/LB)**2)/3) ** 0.5
        self.Dpv = LB * ((1 + (UB/LB)**2) * (1 + (UB/LB)) / 4) ** (1/3)

        ## dW/dDp size distribution to log channel width 
        self.dlogDp = np.log10(UB) - np.log10(LB)


        ## calculate number log
        unit, weight = 2, 1
        self.calculate_file(data, unit, weight)


    def calculate_file(self,data, unit, weight):
        global ndDp
        Q  = 16.666666666  ## расход пробы (см3/сек)  ## sample flow rate
        fi = 1
        ## 
        tz = self.sample_length
        td = data['Dead Time']
    
        ## Концентрация Concentration
        ## number weighted concentration per channel
        #c = data[['z' + str(i) for i in range(0, 16)]]
        #print(c)
        data['kk'] = fi / (Q * (tz-td))
        for i in range(0, 16):
            data['n' + str(i)] = data['z' + str(i)] * data['kk']
        #print(data['kk'])
        n = data[['n' + str(i) for i in range(0, 16)]]
        #print("n:\n", n)

        ## dW/dDp size distribution to channel width in um
        for i in range(0, 16):
            data['ndDp' + str(i)] = data['n' + str(i)] / self.dDp[i]
        ndDp = data[['ndDp' + str(i) for i in range(0, 16)]]
        #print(ndDp)

        ## dW/dDp size distribution to log channel width 
        ## ndlogDp = n / dlogDp
        for i in range(0, 16):
            data['ndlogDp' + str(i)] = data['n' + str(i)] / self.dlogDp[i]
        ndlogDp = data[['ndlogDp' + str(i) for i in range(0, 16)]]
        #print(ndlogDp)

        ## --------------------
        ## 'Number'
        ### Total number concentration
        ## N = sum(n)
        data['N'] = np.sum(n, axis=1)
        #print(data['N'])
        
        # Mode
        # moda_n = Dp[ndDp.idxmax()]
        data['moda_n'] = ndDp.apply(lambda row: self.Dp[list(ndDp.columns).index(row.idxmax())], axis=1)
        #print(data['moda_n'])
        #moda_n = self.Dp[ndDp.idxmax(axis=1).values] 
        #print(moda_n)

        ## Mean
        #mean_n = sum(n * Dp) / N
        arr = data.loc[:,['n' + str(i) for i in range(0, 16)]]
        for i in range(len(arr.columns)):
            arr['n' + str(i)] = arr['n' + str(i)] * self.Dp[i]
        data['mean_n'] = np.sum(arr, axis=1) / data['N']
        #print(data['mean_n'])

        ## Geometric Mean
        ## gmean_n = np.exp(sum(n * np.log(Dp)) / N)
        arr = data.loc[:,['n' + str(i) for i in range(0, 16)]]
        for i in range(len(arr.columns)):
            arr['n' + str(i)] = arr['n' + str(i)] * np.log(self.Dp[i])
        data['gmean_n'] = np.exp(np.sum(arr, axis=1) / data['N'])
        #print(data['gmean_n'])

        ## Geometric Standart Deviation
        ## gsigma_n = np.exp((sum(n * (np.log(Dp) - np.log(gmean_n)) ** 2) / N) ** 0.5)
        arr = data.loc[:,['n' + str(i) for i in range(0, 16)]]
        for i in range(len(arr.columns)):
            arr['n' + str(i)] = arr['n' + str(i)] * (np.log(self.Dp[i]) - np.log(data['gmean_n'])) ** 2
        data['gsigma_n'] = np.exp((np.sum(arr, axis=1) / data['N']) ** 0.5)
        #print(data['gsigma_n'])

        # 
        name = "Number_dW_dlogDp" ##['dw dDp', 'dW dlogDp', 'dW', '%']
        filename = self.prefix + '_' + name + '.csv' 
        print(filename)
        header = 'Sample #,Date,Start Time,Temp(C),Pressure(atm),Rel. Humidity,Errors,Alarm Triggered,Dilution Factor,Dead Time,Median,Mean,Geo. Mean,Mode,Geo. St. Dev.,Total Conc. '
        # "Number" unit = 2 '(#/cm³),Midpoint Diameter'
        header += self.midheader[unit] + ','
        header +=  ','.join([f'{x:.3f}' for x in self.Dp[:]]) + ',,'

        print(header)





global ndDp
# ======================================
dirname = "Satino/"
dirname = "data/2020-12-25/"
## \todo сделать имя папки входным параметром, чтоб можно было указать любую папку и получить полный путь к ней
ops = OPS()
pwd = os.getcwd() + ops.sep
ops.translate_data(pwd + dirname)

Sample file, imp01.O30
Instrument Model,3330
Instrument Serial Number,3330200403
Sample Length (s),60
Alarm Set,No
Dead Time Correction Applied,Yes
Units,Raw Counts
Weight,Number
Refractive Index Applied,No

,,,,,,,,,,,,,,,,LB,0.3,0.44,0.56,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.533,2.95,3.463,5.6,8.032
,,,,,,,,,,,,,,,,UB,0.44,0.56,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.533,2.95,3.463,5.6,8.032,10.0
,,,,,,,,,,,,,,,,LB with RI,0.3,0.44,0.56,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.533,2.95,3.463,5.6,8.032,
,,,,,,,,,,,,,,,,UB with RI,0.44,0.56,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.533,2.95,3.463,5.6,8.032,10.0,
imp01_Number_dW_dlogDp.csv
Sample #,Date,Start Time,Temp(C),Pressure(atm),Rel. Humidity,Errors,Alarm Triggered,Dilution Factor,Dead Time,Median,Mean,Geo. Mean,Mode,Geo. St. Dev.,Total Conc. (#/cm³),Midpoint Diameter,0.370,0.500,0.680,0.900,1.100,1.300,1.500,1.700,1.900,2.100,2.367,2.742,3.207,4.531,6.816,9.016,,


In [50]:
list(ndDp.columns).index('ndDp0')

0

In [ ]:
weights = ['Mass', 'Surface', 'Number']

self.dataheader = 'Sample #,Date,Start Time,Temp(C),Pressure(atm),Rel. Humidity,Errors,Alarm Triggered,Dilution Factor,Dead Time,Median,Mean,Geo. Mean,Mode,Geo. St. Dev.,Total Conc. '
self.midheader = ['(µg/m³),Vol.Wt.Mean Diam.', '(µm²/cm³),Surf.Wt.Mean Diam.', '(#/cm³),Midpoint Diameter']
',,'

'Sample #,Date,Start Time,Temp(C),Pressure(atm),Rel. Humidity,Errors,Alarm Triggered,Dilution Factor,Dead Time,Median,Mean,Geo. Mean,Mode,Geo. St. Dev.,Total Conc. (µg/m³),Vol.Wt.Mean Diam.,0.374,0.502,0.687,0.904,1.103,1.303,1.502,1.702,1.902,2.102,2.370,2.747,3.213,4.614,6.888,9.052,,Vol.Wt.Mean Diam.,0.374,0.502,0.687,0.904,1.103,1.303,1.502,1.702,1.902,2.102,2.370,2.747,3.213,4.614,6.888,9.052,,',
'Sample #,Date,Start Time,Temp(C),Pressure(atm),Rel. Humidity,Errors,Alarm Triggered,Dilution Factor,Dead Time,Median,Mean,Geo. Mean,Mode,Geo. St. Dev.,Total Conc. (µm²/cm³),Surf.Wt.Mean Diam.,0.372,0.501,0.684,0.902,1.102,1.301,1.501,1.701,1.901,2.101,2.368,2.744,3.210,4.573,6.852,9.034,,',
'Sample #,Date,Start Time,Temp(C),Pressure(atm),Rel. Humidity,Errors,Alarm Triggered,Dilution Factor,Dead Time,Median,Mean,Geo. Mean,Mode,Geo. St. Dev.,Total Conc. (#/cm³),Midpoint Diameter,0.370,0.500,0.680,0.900,1.100,1.300,1.500,1.700,1.900,2.100,2.367,2.742,3.207,4.531,6.816,9.016,,'
'Sample #,Date,Start Time,Temp(C),Pressure(atm),Rel. Humidity,Errors,Alarm Triggered,Dilution Factor,Dead Time,Median,Mean,Geo. Mean,Mode,Geo. St. Dev.,Total Conc. (#/cm³),Midpoint Diameter,0.370,0.500,0.680,0.900,1.100,1.300,1.500,1.700,1.900,2.100,2.367,2.742,3.207,4.531,6.816,9.016,,'



['', '0', '1']

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4888ebe5-2b59-4c64-9265-248a80232ba5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>